In [2]:
import datetime as dt
import os
import sys

from cltk.corpus.greek.tlg.parse_tlg_indices import get_id_author
from cltk.corpus.utils.formatter import assemble_tlg_author_filepaths
from nltk.tokenize.punkt import PunktLanguageVars
import pandas

In [3]:
p = PunktLanguageVars()

# Lexical diveristy by author

## Original text with diacritics

### Get status about corpus

In [4]:
t0 = dt.datetime.utcnow()

cleaned_dir = os.path.expanduser('~/cltk_data/greek/text/tlg/plaintext_clean')
dir_contents = os.listdir(cleaned_dir)

corpus_stats = {}

all_tokens_list = []
for doc_count, file in enumerate(dir_contents):
    file_path = os.path.join(cleaned_dir, file)
    with open(file_path) as fo:
        text = fo.read().lower()
    text = ''.join([char for char in text if char not in ['.']])
    tokens = p.word_tokenize(text)
    all_tokens_list += tokens

print('Total author files:', doc_count)
print('Total words:', len(all_tokens_list))
all_tokens_unique = set(all_tokens_list)
print('Total unique words:', len(all_tokens_unique))

corpus_stats = {'doc_count': doc_count, 
               'total_words': len(all_tokens_list),
               'total_unique_words': len(all_tokens_unique)}

print('... finished in {}'.format(dt.datetime.utcnow() - t0))

Total author files: 1822
Total words: 72057716
Total unique words: 1515193
... finished in 0:03:38.839884


In [5]:
df_corpus = pandas.DataFrame(corpus_stats, index=[0])
print(df_corpus)

   doc_count  total_unique_words  total_words
0       1822             1515193     72057716


### Get stats per author

In [6]:
from cltk.corpus.greek.tlg.parse_tlg_indices import get_epithet_of_author

In [7]:
map_id_author = get_id_author()

In [8]:
t0 = dt.datetime.utcnow()

# Words and unique words per author
map_id_word_counts = {}
for file in dir_contents:
    map_word_counts = {}
    file_path = os.path.join(cleaned_dir, file)
    author_id = file[3:-4]
    author = map_id_author[author_id]
    with open(file_path) as fo:
        text = fo.read().lower()
    text = ''.join([char for char in text if char not in ['.']])
    tokens = p.word_tokenize(text)
    map_word_counts['name'] = author
    map_word_counts['epithet'] = get_epithet_of_author(author_id)
    map_word_counts['word_count_all'] = len(tokens)
    map_word_counts['word_count_unique'] = len(set(tokens))
    try:
        lexical_diversity = len(set(tokens)) / len(tokens)
    except ZeroDivisionError:
        lexical_diversity = 0
    map_word_counts['lexical_diversity'] = lexical_diversity
    
    map_id_word_counts[author_id] = map_word_counts
#     print(author)
#     print('    ', 'Total words:', len(tokens))
#     print('    ', 'Total unique words:', len(set(tokens)))
#     print('    ', 'Lexical diversity:', lexical_diversity)

print('... finished in {}'.format(dt.datetime.utcnow() - t0))

... finished in 0:03:44.275525


In [9]:
df_text_counts = pandas.DataFrame(map_id_word_counts).T

In [10]:
df_text_counts

,epithet,lexical_diversity,name,word_count_all,word_count_unique
0001,Epici/-ae,0.34366,Apollonius Rhodius Epic.,39155,13456
0002,Elegiaci,0.398142,Theognis Eleg.,9798,3901
0003,Historici/-ae,0.150405,Thucydides Hist.,150427,22625
0004,Biographi,0.222551,Diogenes Laertius Biogr.,110977,24698
0005,Bucolici,0.439063,Theocritus Bucol.,21719,9536
0006,Tragici,0.19995,Euripides Trag.,184076,36806
0007,Biographi,0.110099,Plutarchus Biogr. et Phil.,1034650,113914
0008,Sophistae,0.151358,Athenaeus Soph.,394588,59724
0009,Lyrici/-ae,0.674388,Sappho Lyr.,3756,2533
0010,Oratores,0.139739,Isocrates Orat.,120603,16853


In [11]:
df_text_counts.to_csv(os.path.expanduser('~/cltk_data/user_data/stats_text_counts.csv'))

### Stats by author

In [12]:
from statistics import mean
from statistics import stdev

In [13]:
author_stats = {}
corpus_word_count_all = []
corpus_word_count_unique = []
corpus_word_lexical_diversity = []
for author_id, map_counts in map_id_word_counts.items():
    corpus_word_count_all.append(map_counts['word_count_all'])
    corpus_word_count_unique.append(map_counts['word_count_unique'])
    corpus_word_lexical_diversity.append(map_counts['lexical_diversity'])

author_stats['mean_words_per_author'] = mean(corpus_word_count_all)
author_stats['standard_deviation_of_words_per_author:'] = stdev(corpus_word_count_all)
author_stats['mean_unique_words_per_author'] = mean(corpus_word_count_unique)
author_stats['standard_deviation_of_unique_words_per_author'] = stdev(corpus_word_count_unique)
author_stats['lexical_diversity_per_author'] = mean(corpus_word_lexical_diversity)
author_stats['standard_deviation_of_lexical_diversity_per_author:'] = stdev(corpus_word_lexical_diversity)

print('Mean words per author:', mean(corpus_word_count_all))
print('Standard deviation of words per author:', stdev(corpus_word_count_all))

print('Mean unique words per author:', mean(corpus_word_count_unique))
print('Standard deviation of unique words per author:', stdev(corpus_word_count_unique))

print('Lexical diversity per author:', mean(corpus_word_lexical_diversity))
print('Standard deviation of lexical diversity per author:', stdev(corpus_word_lexical_diversity))

Mean words per author: 39526.99725726824
Standard deviation of words per author: 174923.28976653758
Mean unique words per author: 5435.820076796489
Standard deviation of unique words per author: 14195.290142159112
Lexical diversity per author: 0.5171187962883808
Standard deviation of lexical diversity per author: 0.2732410961564417


In [14]:
df_authors = pandas.DataFrame(author_stats, index=[0])
df_authors

,lexical_diversity_per_author,mean_unique_words_per_author,mean_words_per_author,standard_deviation_of_lexical_diversity_per_author:,standard_deviation_of_unique_words_per_author,standard_deviation_of_words_per_author:
0,0.517119,5435.820077,39526.997257,0.273241,14195.290142,174923.289767


In [15]:
df_authors.to_csv(os.path.expanduser('~/cltk_data/user_data/stats_authors.csv'))

### Get stats about epithets

In [16]:
from collections import defaultdict
import datetime as dt
from cltk.corpus.greek.tlg.parse_tlg_indices import get_epithets

In [17]:
list_epithets = get_epithets()

In [18]:
t0 = dt.datetime.utcnow()

map_epithet_counts_all = defaultdict(list)
map_epithet_counts_unique = defaultdict(list)
map_epithet_lexical_diversity = defaultdict(list)
for file in dir_contents:
    map_word_counts = defaultdict(list)
    file_path = os.path.join(cleaned_dir, file)
    author_id = file[3:-4]
    author = map_id_author[author_id]
    with open(file_path) as fo:
        text = fo.read().lower()
    text = ''.join([char for char in text if char not in ['.']])
    tokens = p.word_tokenize(text)
    try:
        lexical_diversity = len(set(tokens)) / len(tokens)
    except ZeroDivisionError:
        lexical_diversity = 0
    epithet = get_epithet_of_author(author_id)

    map_epithet_counts_all[epithet].append(len(tokens))
    map_epithet_counts_unique[epithet].append(len(set(tokens)))
    map_epithet_lexical_diversity[epithet].append(lexical_diversity)

print('... finished in {}'.format(dt.datetime.utcnow() - t0))

... finished in 0:03:44.608805


In [24]:
from statistics import StatisticsError

epithet_lexical_diversity_tuples = []
epithet_scores = {}
for epithet, counts in map_epithet_counts_all.items():
    print(epithet)
    print('    Mean of word counts per author:', mean(counts))
    try:
        wc_standard_deviation = stdev(counts)
    except StatisticsError:
        wc_standard_deviation = 0
    print('    Standard deviation of word counts per author:', wc_standard_deviation)
    
    uniques_list = map_epithet_counts_unique[epithet]
    print('    Mean of unique word counts per author:', mean(uniques_list))
    try:
        uniques_standard_deviation = stdev(uniques_list)
    except StatisticsError:
        uniques_standard_deviation = 0
    print('    Standard deviation of unique word counts per author:', uniques_standard_deviation)

    lexical_diversity_list = map_epithet_lexical_diversity[epithet]
    print('    Mean of lexical diversity per author:', mean(lexical_diversity_list))
    try:
        ld_standard_deviation = stdev(lexical_diversity_list)
    except StatisticsError:
        ld_standard_deviation = 0
    print('    Standard deviation of lexical diversity:', ld_standard_deviation)
    
    epithet_lexical_diversity_tuples.append((epithet, mean(lexical_diversity_list)))

    tmp_scores = {}
    tmp_scores['mean_of_word_counts_ per_author'] = mean(counts)
    tmp_scores['standard_deviation_of_word_counts_per_author'] = wc_standard_deviation
    tmp_scores['mean_of_unique_word_counts_per_author'] = mean(uniques_list)
    tmp_scores['standard_deviation_of_unique_word_counts_per_author'] = uniques_standard_deviation
    tmp_scores['mean_of_lexical_diversity_per_author'] = mean(lexical_diversity_list)
    tmp_scores['standard_deviation_of_lexical_diversity'] = ld_standard_deviation
    epithet_scores[epithet] = tmp_scores

Mechanici
    Mean of word counts per author: 27850.714285714286
    Standard deviation of word counts per author: 58272.68047926142
    Mean of unique word counts per author: 3668.4285714285716
    Standard deviation of unique word counts per author: 4857.279000192832
    Mean of lexical diversity per author: 0.30923266180627396
    Standard deviation of lexical diversity: 0.1190888649052873
Mimographi
    Mean of word counts per author: 2752
    Standard deviation of word counts per author: 3008.032247167573
    Mean of unique word counts per author: 1479
    Standard deviation of unique word counts per author: 1438.2551929334377
    Mean of lexical diversity per author: 0.6253901209264193
    Standard deviation of lexical diversity: 0.16095147451789965
Hymnographi
    Mean of word counts per author: 126901
    Standard deviation of word counts per author: 0
    Mean of unique word counts per author: 24999
    Standard deviation of unique word counts per author: 0
    Mean of lexical

In [25]:
# sort epithets by lexical diversity
sorted(epithet_lexical_diversity_tuples, key=lambda x: x[1], reverse=True)

[('Choliambographi', 1.0),
 ('Epigrammatici/-ae', 0.8487669388459786),
 ('Elegiaci', 0.7829407154009786),
 ('Iambici', 0.7662045014476172),
 ('Lyrici/-ae', 0.7476551963199876),
 ('Poetae Didactici', 0.7354838709677419),
 ('Poetae Medici', 0.7121893719806763),
 ('Mimographi', 0.6253901209264193),
 ('Tragici', 0.6186934024311059),
 ('Gnostici', 0.614900854930397),
 ('Poetae Philosophi', 0.6119466967099234),
 ('Gnomici', 0.6072317526129395),
 ('Historici/-ae', 0.602783132461963),
 ('Parodii', 0.6014235088742101),
 ('Epici/-ae', 0.5901755381170039),
 ('Poetae', 0.5715269592682862),
 ('Paradoxographi', 0.5526884575900334),
 ('Bucolici', 0.5318337561964496),
 (None, 0.5262500789551204),
 ('Comici', 0.5163583294667708),
 ('Alchemistae', 0.4830936632872289),
 ('Philosophici/-ae', 0.4755907773618297),
 ('Geographi', 0.46230688215853827),
 ('Lexicographi', 0.45092513391608313),
 ('Mythographi', 0.44350822261989964),
 ('Grammatici', 0.4383668023216881),
 ('Apologetici', 0.4362444959064383),
 ('Pe

In [26]:
pandas.DataFrame(epithet_lexical_diversity_tuples)

,0,1
0,Mechanici,0.309233
1,Mimographi,0.625390
2,Hymnographi,0.196996
3,Philologi,0.288283
4,Iambici,0.766205
5,Poetae,0.571527
6,Medici,0.339495
7,Scriptores Ecclesiastici,0.320798
8,Periegetae,0.423661
9,Paradoxographi,0.552688


In [27]:
df_epithet_scores = pandas.DataFrame(epithet_scores).T
df_epithet_scores

,mean_of_lexical_diversity_per_author,mean_of_unique_word_counts_per_author,mean_of_word_counts_ per_author,standard_deviation_of_lexical_diversity,standard_deviation_of_unique_word_counts_per_author,standard_deviation_of_word_counts_per_author
Mimographi,0.625390,1479.000000,2752.000000,0.160951,1438.255193,3008.032247
Philologi,0.288283,29787.111111,260571.000000,0.130476,48661.146589,554261.013121
Iambici,0.766205,423.500000,610.500000,0.268881,658.157656,966.676205
Poetae,0.571527,2542.923077,8495.230769,0.350696,5088.694101,23588.759892
Lexicographi,0.450925,65895.750000,265077.250000,0.348227,46740.193477,235698.556551
Medici,0.339495,14082.105263,132242.078947,0.234760,26032.439766,414574.020743
Scriptores Ecclesiastici,0.320798,19739.913793,203310.413793,0.190431,29989.263181,587654.370329
Periegetae,0.423661,5329.444444,28730.000000,0.143773,10078.459953,71070.642007
Paradoxographi,0.552688,1278.000000,3216.444444,0.168156,1403.297545,4577.134341
Scriptores Erotici,0.403190,6774.875000,24547.375000,0.180467,6261.752001,26112.565306


In [28]:
df_epithet_scores.to_csv(os.path.expanduser('~/cltk_data/user_data/stats_epithet.csv'))